Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
# data files are in root repository
pickle_file = '../notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
              / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

#### First we work on logistic regression

 - We use the minibatch implementation from assignment 2.

In [9]:
# Create TensorFlow graph

batch_size = 128
# regularisation constant
gamma = 0.01

graph1 = tf.Graph()
with graph1.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights = tf.Variable(
         tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    
    # tf.reduce_mean because we take the average cross entropy over the batch.
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    # add regularisation to loss
    # notes: regularise both weights and biases
    loss = loss + gamma * (
    tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases)
    )
        
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [6]:
# run tensorFlow graph.

num_steps = 3001

with tf.Session(graph=graph1) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 44.337463
Minibatch accuracy: 14.1%
Validation accuracy: 15.0%
Minibatch loss at step 500: 0.793702
Minibatch accuracy: 86.7%
Validation accuracy: 81.4%
Minibatch loss at step 1000: 0.802103
Minibatch accuracy: 83.6%
Validation accuracy: 81.3%
Minibatch loss at step 1500: 0.631764
Minibatch accuracy: 85.2%
Validation accuracy: 81.5%
Minibatch loss at step 2000: 0.848513
Minibatch accuracy: 78.1%
Validation accuracy: 79.8%
Minibatch loss at step 2500: 0.717123
Minibatch accuracy: 82.8%
Validation accuracy: 81.7%
Minibatch loss at step 3000: 0.787688
Minibatch accuracy: 78.1%
Validation accuracy: 77.7%
Test accuracy: 83.9%


#### Now let's work on a neural network with a hidden layer

- We use the example from assignment 2:

In [12]:
batch_size = 128
hidden_nodes = 1024
# regularisation constant
gamma = 0.01

graph2 = tf.Graph()
with graph2.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables - Network Construction!
    # Matrix Dimensions:
    # 1st argument has dimensions coming from previous layer
    # 2nd argument has dimensions going to the next layer == dim(bias)
    # We construct the variables representing the hidden layer:
    weights1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden_nodes]))
    biases1 = tf.Variable(tf.zeros([hidden_nodes]))
    # We construct the variables representing the output layer:
    weights2 = tf.Variable(
        tf.truncated_normal([hidden_nodes, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    hidden_layer = tf.matmul(tf_train_dataset, weights1) + biases1
    logits = tf.matmul(hidden_layer, weights2) + biases2
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(
            labels=tf_train_labels, logits=logits))
    # add regularisation for all weights.
    loss = loss + gamma * (
    tf.nn.l2_loss(weights1) + tf.nn.l2_loss(biases1) + tf.nn.l2_loss(weights2) + tf.nn.l2_loss(biases2))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    # Predict for training:
    train_prediction = tf.nn.softmax(logits)
    
    # Create Validation graph
    hidden_layer_val = tf.matmul(tf_valid_dataset, weights1) + biases1
    logits_val = tf.matmul(hidden_layer_val, weights2) + biases2
    # Predict for validation
    valid_prediction = tf.nn.softmax(logits_val)
    
    # Create Test graph
    hidden_layer_test = tf.matmul(tf_test_dataset, weights1) + biases1
    logits_test = tf.matmul(hidden_layer_test, weights2) + biases2
    # Predict for test
    test_prediction = tf.nn.softmax(logits_test)

In [8]:
num_steps = 3001

with tf.Session(graph=graph2) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
            
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3777.592285
Minibatch accuracy: 7.0%
Validation accuracy: 33.5%
Minibatch loss at step 500: 20.650795
Minibatch accuracy: 85.9%
Validation accuracy: 79.9%
Minibatch loss at step 1000: 0.989282
Minibatch accuracy: 82.0%
Validation accuracy: 81.3%
Minibatch loss at step 1500: 0.666988
Minibatch accuracy: 85.9%
Validation accuracy: 81.4%
Minibatch loss at step 2000: 0.862262
Minibatch accuracy: 78.9%
Validation accuracy: 79.6%
Minibatch loss at step 2500: 0.751192
Minibatch accuracy: 82.8%
Validation accuracy: 81.7%
Minibatch loss at step 3000: 0.820589
Minibatch accuracy: 80.5%
Validation accuracy: 76.9%
Test accuracy: 82.6%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

#### First we work on logistic regression

In [10]:
### NOTE: Rerun graph1 build step before running ###

# run tensorFlow graph for logistic regression.

num_steps = 3001

with tf.Session(graph=graph1) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: restrict offset to [1, 500]
        offset = np.random.choice(list(range(1, 501)))
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 51.434483
Minibatch accuracy: 7.8%
Validation accuracy: 11.0%
Minibatch loss at step 500: 0.619656
Minibatch accuracy: 96.9%
Validation accuracy: 75.7%
Minibatch loss at step 1000: 0.352243
Minibatch accuracy: 98.4%
Validation accuracy: 76.7%
Minibatch loss at step 1500: 0.302185
Minibatch accuracy: 100.0%
Validation accuracy: 76.7%
Minibatch loss at step 2000: 0.288289
Minibatch accuracy: 100.0%
Validation accuracy: 76.8%
Minibatch loss at step 2500: 0.324579
Minibatch accuracy: 98.4%
Validation accuracy: 76.4%
Minibatch loss at step 3000: 0.329259
Minibatch accuracy: 100.0%
Validation accuracy: 76.7%
Test accuracy: 83.2%


#### Now let's work on a neural network with a hidden layer

In [13]:
### NOTE: Rerun graph1 build step before running ###
num_steps = 3001

with tf.Session(graph=graph2) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: restrict offset to [1, 500]
        offset = np.random.choice(list(range(1, 501)))
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
            
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3579.202881
Minibatch accuracy: 6.2%
Validation accuracy: 23.8%
Minibatch loss at step 500: 21.021086
Minibatch accuracy: 100.0%
Validation accuracy: 74.7%
Minibatch loss at step 1000: 0.425974
Minibatch accuracy: 100.0%
Validation accuracy: 76.4%
Minibatch loss at step 1500: 0.311039
Minibatch accuracy: 100.0%
Validation accuracy: 76.0%
Minibatch loss at step 2000: 0.358084
Minibatch accuracy: 99.2%
Validation accuracy: 76.2%
Minibatch loss at step 2500: 0.324903
Minibatch accuracy: 100.0%
Validation accuracy: 76.2%
Minibatch loss at step 3000: 0.367288
Minibatch accuracy: 97.7%
Validation accuracy: 75.7%
Test accuracy: 81.9%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

#### Introducing dropout for the hidden layer

In [14]:
batch_size = 128
hidden_nodes = 1024

graph3 = tf.Graph()
with graph3.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables - Network Construction!
    # Matrix Dimensions:
    # 1st argument has dimensions coming from previous layer
    # 2nd argument has dimensions going to the next layer == dim(bias)
    # We construct the variables representing the hidden layer:
    weights1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden_nodes]))
    biases1 = tf.Variable(tf.zeros([hidden_nodes]))
    # We construct the variables representing the output layer:
    weights2 = tf.Variable(
        tf.truncated_normal([hidden_nodes, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    hidden_layer = tf.matmul(tf_train_dataset, weights1) + biases1
    # introduce dropout
    keep_prob = tf.placeholder(tf.float32)
    hidden_layer_d = tf.nn.dropout(hidden_layer, keep_prob)
    
    # logits for prediction
    logits = tf.matmul(hidden_layer, weights2) + biases2
    # logits_d for training with dropout
    logits_d = tf.matmul(hidden_layer_d, weights2) + biases2
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(
            labels=tf_train_labels, logits=logits_d))
    
    # add regularisation for all weights.
    regconst = tf.placeholder(tf.float32)
    loss = loss + regconst * (
    tf.nn.l2_loss(weights1) + tf.nn.l2_loss(biases1) + tf.nn.l2_loss(weights2) + tf.nn.l2_loss(biases2))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    # Predict for training:
    train_prediction = tf.nn.softmax(logits)
    
    # Create Validation graph
    hidden_layer_val = tf.matmul(tf_valid_dataset, weights1) + biases1
    logits_val = tf.matmul(hidden_layer_val, weights2) + biases2
    # Predict for validation
    valid_prediction = tf.nn.softmax(logits_val)
    
    # Create Test graph
    hidden_layer_test = tf.matmul(tf_test_dataset, weights1) + biases1
    logits_test = tf.matmul(hidden_layer_test, weights2) + biases2
    # Predict for test
    test_prediction = tf.nn.softmax(logits_test)

In [15]:
num_steps = 3001
# dropout layer keep probability
keep_probl = 0.5 # cannot have the same name as graph variable!
# regularisation constant
gamma = 0.01

with tf.Session(graph=graph3) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data,
                     tf_train_labels : batch_labels,
                     regconst : gamma,
                     keep_prob : keep_probl}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
            
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3738.199219
Minibatch accuracy: 10.2%
Validation accuracy: 38.7%
Minibatch loss at step 500: 22.035246
Minibatch accuracy: 82.8%
Validation accuracy: 77.2%
Minibatch loss at step 1000: 1.054348
Minibatch accuracy: 82.0%
Validation accuracy: 80.4%
Minibatch loss at step 1500: 0.715341
Minibatch accuracy: 85.2%
Validation accuracy: 81.0%
Minibatch loss at step 2000: 0.921868
Minibatch accuracy: 78.1%
Validation accuracy: 78.4%
Minibatch loss at step 2500: 0.803455
Minibatch accuracy: 82.8%
Validation accuracy: 81.1%
Minibatch loss at step 3000: 0.879746
Minibatch accuracy: 79.7%
Validation accuracy: 75.5%
Test accuracy: 80.9%


#### Restricting training data

In [16]:
num_steps = 3001
# dropout layer keep probability
keep_probl = 0.5 # cannot have the same name as graph variable!
# regularisation constant
gamma = 0.01

with tf.Session(graph=graph3) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: restrict offset to [1, 500]
        offset = np.random.choice(list(range(1, 501)))
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data,
                     tf_train_labels : batch_labels,
                     regconst : gamma,
                     keep_prob : keep_probl}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
            
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3793.855469
Minibatch accuracy: 12.5%
Validation accuracy: 38.0%
Minibatch loss at step 500: 21.905928
Minibatch accuracy: 100.0%
Validation accuracy: 74.9%
Minibatch loss at step 1000: 0.474580
Minibatch accuracy: 100.0%
Validation accuracy: 76.0%
Minibatch loss at step 1500: 0.386491
Minibatch accuracy: 99.2%
Validation accuracy: 76.0%
Minibatch loss at step 2000: 0.309371
Minibatch accuracy: 100.0%
Validation accuracy: 76.2%
Minibatch loss at step 2500: 0.388107
Minibatch accuracy: 97.7%
Validation accuracy: 74.5%
Minibatch loss at step 3000: 0.430854
Minibatch accuracy: 97.7%
Validation accuracy: 73.0%
Test accuracy: 79.5%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


### 1. We start by increasing the training step on the regularised with dropout 1 hidden_layer network.

In [17]:
num_steps = 8001
# dropout layer keep probability
keep_probl = 0.5 # cannot have the same name as graph variable!
# regularisation constant
gamma = 0.01

with tf.Session(graph=graph3) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data,
                     tf_train_labels : batch_labels,
                     regconst : gamma,
                     keep_prob : keep_probl}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
            
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3926.123535
Minibatch accuracy: 10.2%
Validation accuracy: 32.0%
Minibatch loss at step 500: 21.765230
Minibatch accuracy: 84.4%
Validation accuracy: 78.6%
Minibatch loss at step 1000: 1.071986
Minibatch accuracy: 82.0%
Validation accuracy: 80.1%
Minibatch loss at step 1500: 0.725598
Minibatch accuracy: 85.2%
Validation accuracy: 80.9%
Minibatch loss at step 2000: 0.913908
Minibatch accuracy: 78.1%
Validation accuracy: 78.3%
Minibatch loss at step 2500: 0.844660
Minibatch accuracy: 81.2%
Validation accuracy: 81.3%
Minibatch loss at step 3000: 0.871586
Minibatch accuracy: 80.5%
Validation accuracy: 75.8%
Minibatch loss at step 3500: 0.896901
Minibatch accuracy: 79.7%
Validation accuracy: 80.5%
Minibatch loss at step 4000: 1.069300
Minibatch accuracy: 74.2%
Validation accuracy: 78.3%
Minibatch loss at step 4500: 0.985554
Minibatch accuracy: 78.9%
Validation accuracy: 79.9%
Minibatch loss at step 5000: 0.911632
Minibatch accuracy: 77.3%
Validation acc

### 2. Let's increase regularisation.

In [18]:
num_steps = 8001
# dropout layer keep probability
keep_probl = 0.5 # cannot have the same name as graph variable!
# regularisation constant
gamma = 0.03

with tf.Session(graph=graph3) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data,
                     tf_train_labels : batch_labels,
                     regconst : gamma,
                     keep_prob : keep_probl}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
            
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 10099.887695
Minibatch accuracy: 9.4%
Validation accuracy: 40.9%
Minibatch loss at step 500: 0.882456
Minibatch accuracy: 85.9%
Validation accuracy: 80.6%
Minibatch loss at step 1000: 1.110740
Minibatch accuracy: 80.5%
Validation accuracy: 79.3%
Minibatch loss at step 1500: 0.894144
Minibatch accuracy: 83.6%
Validation accuracy: 80.5%
Minibatch loss at step 2000: 1.054620
Minibatch accuracy: 78.1%
Validation accuracy: 77.8%
Minibatch loss at step 2500: 1.026155
Minibatch accuracy: 80.5%
Validation accuracy: 80.5%
Minibatch loss at step 3000: 1.021120
Minibatch accuracy: 81.2%
Validation accuracy: 75.2%
Minibatch loss at step 3500: 1.063710
Minibatch accuracy: 80.5%
Validation accuracy: 80.0%
Minibatch loss at step 4000: 1.215300
Minibatch accuracy: 73.4%
Validation accuracy: 78.4%
Minibatch loss at step 4500: 1.115122
Minibatch accuracy: 76.6%
Validation accuracy: 77.7%
Minibatch loss at step 5000: 1.093077
Minibatch accuracy: 77.3%
Validation accu

#### Results:

Increasing regularisation above 0.01 didn't increase performance!

### 3. Let's double the width of the hidden layer:

In [19]:
batch_size = 128
hidden_nodes = 1024

graph4 = tf.Graph()
with graph4.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables - Network Construction!
    # Matrix Dimensions:
    # 1st argument has dimensions coming from previous layer
    # 2nd argument has dimensions going to the next layer == dim(bias)
    # We construct the variables representing the hidden layer:
    weights1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden_nodes]))
    biases1 = tf.Variable(tf.zeros([hidden_nodes]))
    # We construct the variables representing the output layer:
    weights2 = tf.Variable(
        tf.truncated_normal([hidden_nodes, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    hidden_layer = tf.matmul(tf_train_dataset, weights1) + biases1
    # introduce dropout
    keep_prob = tf.placeholder(tf.float32)
    hidden_layer_d = tf.nn.dropout(hidden_layer, keep_prob)
    
    # logits for prediction
    logits = tf.matmul(hidden_layer, weights2) + biases2
    # logits_d for training with dropout
    logits_d = tf.matmul(hidden_layer_d, weights2) + biases2
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(
            labels=tf_train_labels, logits=logits_d))
    
    # add regularisation for all weights.
    regconst = tf.placeholder(tf.float32)
    loss = loss + regconst * (
    tf.nn.l2_loss(weights1) + tf.nn.l2_loss(biases1) + tf.nn.l2_loss(weights2) + tf.nn.l2_loss(biases2))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    # Predict for training:
    train_prediction = tf.nn.softmax(logits)
    
    # Create Validation graph
    hidden_layer_val = tf.matmul(tf_valid_dataset, weights1) + biases1
    logits_val = tf.matmul(hidden_layer_val, weights2) + biases2
    # Predict for validation
    valid_prediction = tf.nn.softmax(logits_val)
    
    # Create Test graph
    hidden_layer_test = tf.matmul(tf_test_dataset, weights1) + biases1
    logits_test = tf.matmul(hidden_layer_test, weights2) + biases2
    # Predict for test
    test_prediction = tf.nn.softmax(logits_test)

In [20]:
num_steps = 8001
# dropout layer keep probability
keep_probl = 0.5 # cannot have the same name as graph variable!
# regularisation constant
gamma = 0.01

with tf.Session(graph=graph4) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data,
                     tf_train_labels : batch_labels,
                     regconst : gamma,
                     keep_prob : keep_probl}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
            
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 7265.819824
Minibatch accuracy: 12.5%
Validation accuracy: 36.1%
Minibatch loss at step 500: 45.193207
Minibatch accuracy: 79.7%
Validation accuracy: 76.7%
Minibatch loss at step 1000: 1.189846
Minibatch accuracy: 82.0%
Validation accuracy: 79.7%
Minibatch loss at step 1500: 0.717308
Minibatch accuracy: 84.4%
Validation accuracy: 81.0%
Minibatch loss at step 2000: 0.925651
Minibatch accuracy: 78.1%
Validation accuracy: 77.9%
Minibatch loss at step 2500: 0.821360
Minibatch accuracy: 82.8%
Validation accuracy: 81.2%
Minibatch loss at step 3000: 0.869665
Minibatch accuracy: 81.2%
Validation accuracy: 75.4%
Minibatch loss at step 3500: 0.899065
Minibatch accuracy: 80.5%
Validation accuracy: 80.6%
Minibatch loss at step 4000: 1.005849
Minibatch accuracy: 74.2%
Validation accuracy: 80.0%
Minibatch loss at step 4500: 0.972044
Minibatch accuracy: 78.1%
Validation accuracy: 79.0%
Minibatch loss at step 5000: 0.921037
Minibatch accuracy: 76.6%
Validation acc

The accuracy of the network did not significantly increase with the increase of the hidden nodes.

### 4. Let's try 2 hidden layers of size 1024

In [76]:
batch_size = 128
hidden_nodes1 = 1024
hidden_nodes2 = 1024

graph5 = tf.Graph()
with graph5.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables - Network Construction!
    # Matrix Dimensions:
    # 1st argument has dimensions coming from previous layer
    # 2nd argument has dimensions going to the next layer == dim(bias)
    # We construct the variables representing the 1st hidden layer:
    weights1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden_nodes1]))
    biases1 = tf.Variable(tf.zeros([hidden_nodes1]))
    # We construct the variables representing the 2nd hidden layer:
    weights2 = tf.Variable(
    tf.random_normal([hidden_nodes1, hidden_nodes2]))
    biases2 = tf.Variable(tf.zeros([hidden_nodes2]))
    # We construct the variables representing the output layer:
    weights3 = tf.Variable(
        tf.truncated_normal([hidden_nodes2, num_labels]))
    biases3 = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    hidden_layer1 = tf.matmul(tf_train_dataset, weights1) + biases1
    hidden_layer2 = tf.matmul(hidden_layer1, weights2) + biases2
    # introduce dropout
    keep_prob = tf.placeholder(tf.float32)
    hidden_layer1_d = tf.nn.dropout(hidden_layer1, keep_prob)
    hidden_layer2_d = tf.nn.dropout(hidden_layer2, keep_prob)
    
    # logits for prediction
    logits = tf.matmul(hidden_layer2, weights3) + biases3
    # logits_d for training with dropout
    logits_d = tf.matmul(hidden_layer2_d, weights3) + biases3
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(
            labels=tf_train_labels, logits=logits_d))
    
    # add regularisation for all weights.
    regconst = tf.placeholder(tf.float32)
    loss = loss + regconst * (
    tf.nn.l2_loss(weights1) + tf.nn.l2_loss(biases1) 
        + tf.nn.l2_loss(weights2) + tf.nn.l2_loss(biases2)
        + tf.nn.l2_loss(weights3) + tf.nn.l2_loss(biases3))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.003).minimize(loss)

    # Predictions for the training, validation, and test data.
    # Predict for training:
    train_prediction = tf.nn.softmax(logits)
    
    # Create Validation graph
    hidden_layer1_val = tf.matmul(tf_valid_dataset, weights1) + biases1
    hidden_layer2_val = tf.matmul(hidden_layer1_val, weights2) + biases2
    logits_val = tf.matmul(hidden_layer2_val, weights3) + biases3
    # Predict for validation
    valid_prediction = tf.nn.softmax(logits_val)
    
    # Create Test graph
    hidden_layer1_test = tf.matmul(tf_test_dataset, weights1) + biases1
    hidden_layer2_test = tf.matmul(hidden_layer1_test, weights2) + biases2
    logits_test = tf.matmul(hidden_layer2_test, weights3) + biases3
    # Predict for test
    test_prediction = tf.nn.softmax(logits_test)
    
    ## Notes:
    # 2 hidden layers cause instability in gradient backpropagation
    # The solution is a combination of appropriate initialisation and learning rate.
    # Here the learning rate was reduced.

In [78]:
num_steps = 24001
# dropout layer keep probability
keep_probl = 0.05 # cannot have the same name as graph variable!
# regularisation constant
gamma = 0.01

with tf.Session(graph=graph5) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data,
                     tf_train_labels : batch_labels,
                     regconst : gamma,
                     keep_prob : keep_probl}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 2000 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
            
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 72049.484375
Minibatch accuracy: 7.8%
Validation accuracy: 40.8%
Minibatch loss at step 2000: 20397.914062
Minibatch accuracy: 78.1%
Validation accuracy: 80.1%
Minibatch loss at step 4000: 11478.924805
Minibatch accuracy: 78.9%
Validation accuracy: 81.3%
Minibatch loss at step 6000: 6038.848633
Minibatch accuracy: 86.7%
Validation accuracy: 81.2%
Minibatch loss at step 8000: 5298.792480
Minibatch accuracy: 79.7%
Validation accuracy: 81.9%
Minibatch loss at step 10000: 4220.005371
Minibatch accuracy: 81.2%
Validation accuracy: 81.7%
Minibatch loss at step 12000: 3494.538818
Minibatch accuracy: 78.9%
Validation accuracy: 82.1%
Minibatch loss at step 14000: 2904.748779
Minibatch accuracy: 81.2%
Validation accuracy: 82.0%
Minibatch loss at step 16000: 2535.443604
Minibatch accuracy: 85.2%
Validation accuracy: 81.7%
Minibatch loss at step 18000: 2149.887939
Minibatch accuracy: 83.6%
Validation accuracy: 82.0%
Minibatch loss at step 20000: 1887.889282
Mi

The network accuracy increased slightly but not significantly.